In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)
    return images, labels

In [3]:
x_train, y_train = load_mnist('',kind='train')
x_test, y_test = load_mnist('',kind='t10k')

In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [22]:
model = Sequential([
    tf.keras.Input(shape=(784,1)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
])
chanDim = 1
# first CONV => RELU => CONV => RELU => POOL layer set
model.add(layers.Conv2D(32, (3, 3), padding="same", input_shape=(28, 28, 1)))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.Conv2D(32, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
# second CONV => RELU => CONV => RELU => POOL layer set
model.add(layers.Conv2D(64, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.Conv2D(64, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
# first (and only) set of FC => RELU layers
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
# softmax classifier
model.add(layers.Dense(10))
model.add(layers.Activation("softmax"))

In [23]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 28, 32)        112       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 28, 28, 32)       

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train, epochs=7)

Epoch 1/7
1875/1875 [==============================] - 256s 136ms/step - loss: 0.4767 - accuracy: 0.8342
Epoch 2/7
1875/1875 [==============================] - 346s 185ms/step - loss: 0.3063 - accuracy: 0.8906
Epoch 3/7
1875/1875 [==============================] - 438s 234ms/step - loss: 0.2728 - accuracy: 0.9030
Epoch 4/7
1875/1875 [==============================] - 440s 235ms/step - loss: 0.2567 - accuracy: 0.9073
Epoch 5/7
1875/1875 [==============================] - 439s 234ms/step - loss: 0.2322 - accuracy: 0.9168
Epoch 6/7
1875/1875 [==============================] - 438s 233ms/step - loss: 0.2178 - accuracy: 0.9208
Epoch 7/7
1875/1875 [==============================] - 437s 233ms/step - loss: 0.2025 - accuracy: 0.9273


In [26]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1,batch_size = 1)
print('\nTest accuracy:', test_acc)

10000/10000 [==============================] - 59s 6ms/step - loss: 0.2040 - accuracy: 0.9258

Test accuracy: 0.9258000254631042
